In [1]:
import os
import torch
import skimage
import pywt
import scipy.io
import scipy.signal
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from scipy import stats
from einops import reduce, rearrange, repeat
from npeet import entropy_estimators as ee
from torch.optim.lr_scheduler import StepLR
from scipy.fft import rfft, rfftfreq, ifft
from einops import rearrange
from torch_geometric.data import InMemoryDataset, Data, DataLoader
from Electrodes import Electrodes
from tqdm import tqdm

In [2]:
class DEAPDatasetEEGFeatures(InMemoryDataset):
  def __init__(self, root, raw_dir, processed_dir, feature='de', transform=None, pre_transform=None,include_edge_attr = True, undirected_graphs = True, add_global_connections=True, participant_from=1, participant_to=32, n_videos=40):
      self._raw_dir = raw_dir
      self._processed_dir = processed_dir
      self.participant_from = participant_from
      self.participant_to = participant_to
      self.n_videos = n_videos
      self.feature = feature
      # Whether or not to include edge_attr in the dataset
      self.include_edge_attr = include_edge_attr
      # If true there will be 1024 links as opposed to 528
      self.undirected_graphs = undirected_graphs
      # Instantiate class to handle electrode positions
      print('Using global connections' if add_global_connections else 'Not using global connections')
      self.electrodes = Electrodes(add_global_connections, expand_3d = False)
      super(DEAPDatasetEEGFeatures, self).__init__(root, transform, pre_transform)
      self.data, self.slices = torch.load(self.processed_paths[0])
      
  @property
  def raw_dir(self):
      return f'{self.root}/{self._raw_dir}'

  @property
  def processed_dir(self):
      return f'{self.root}/{self._processed_dir}'

  @property
  def raw_file_names(self):
      raw_names = [f for f in os.listdir(self.raw_dir)]
      raw_names.sort()
      return raw_names

  @property
  def processed_file_names(self):
      if not os.path.exists(self.processed_dir):
        os.makedirs(self.processed_dir)
      file_name = f'{self.participant_from}-{self.participant_to}' if self.participant_from is not self.participant_to else f'{self.participant_from}'
      return [f'deap_processed_graph.{file_name}_{self.feature}.dataset']

  def process(self):
        # Number of nodes per graph
        n_nodes = len(self.electrodes.positions_3d)
        

        if self.undirected_graphs:
            source_nodes, target_nodes = np.repeat(np.arange(0,n_nodes),n_nodes), np.tile(np.arange(0,n_nodes),n_nodes)
        else:
            source_nodes, target_nodes = np.tril_indices(n_nodes,n_nodes)
        
        edge_attr = self.electrodes.adjacency_matrix[source_nodes,target_nodes]
        
        # Remove zero weight links
        mask = np.ma.masked_not_equal(edge_attr, 0).mask
        edge_attr,source_nodes,target_nodes = edge_attr[mask], source_nodes[mask], target_nodes[mask]

        edge_attr, edge_index = torch.FloatTensor(edge_attr), torch.tensor([source_nodes,target_nodes], dtype=torch.long)
        
        # Expand edge_index and edge_attr to match windows
        e_edge_index = edge_index.clone()
        e_edge_attr = edge_attr.clone()
        number_of_graphs = 4
        for i in range(number_of_graphs-1):
            a = edge_index + e_edge_index.max() + 1
            e_edge_index = torch.cat([e_edge_index,a],dim=1)
            e_edge_attr = torch.cat([e_edge_attr,edge_attr],dim=0)

        print(f'Number of graphs per video: {number_of_graphs}')
        # List of graphs that will be written to file
        data_list = []
        pbar = tqdm(range(self.participant_from,self.participant_to+1))
        for participant_id in pbar:
            raw_name = [e for e in self.raw_file_names if str(participant_id).zfill(2) in e][0]
            pbar.set_description(raw_name)
            # Load raw file as np array
            participant_data = scipy.io.loadmat(f'{self.raw_dir}/{raw_name}')
            signal_data = torch.FloatTensor(remove_baseline_mean(participant_data['data'][:,:32,:]))
#             signal_data = torch.FloatTensor()
            processed = []
            for i, video in enumerate(signal_data[:self.n_videos,:,:]):
                if self.feature == 'wav':
                    node_features = process_video_wavelet(video)
                else:
                    node_features = process_video(video, feature=self.feature)
                data = Data(x=torch.FloatTensor(node_features),edge_attr=e_edge_attr,edge_index=e_edge_index, y=torch.FloatTensor([participant_data['labels'][i]])) if self.include_edge_attr else Data(x=torch.FloatTensor(node_features), edge_index=e_edge_index, y=torch.FloatTensor([participant_data['labels'][i]]))
                data_list.append(data) 
               
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [3]:
def calculate_de(window):
    return ee.entropy(window.reshape(-1,1), k=2)
# Input: Video with shape (32,7680)
# Output: Graph node features with shape (5*32, 59) -> 5 graphs with 32 nodes each with 59 features each
def process_video(video, feature='psd'):
    # Transform to frequency domain
    fft_vals = np.fft.rfft(video, axis=-1)
     # Get frequencies for amplitudes in Hz
    samplingFrequency = 128
    fft_freq = np.fft.rfftfreq(video.shape[-1], 1.0/samplingFrequency)
    # Delta, Theta, Alpha, Beta, Gamma
    bands = [(0,4),(4,8),(8,12),(12,30),(30,45)]
    
    band_mask = np.array([np.logical_or(fft_freq < f, fft_freq > t) for f,t in bands])
    band_mask = repeat(band_mask,'a b -> a c b', c=32)
    band_data = np.array(fft_vals)
    band_data = repeat(band_data,'a b -> c a b', c=5)
     
    band_data[band_mask] = 0
    
    band_data = np.fft.irfft(band_data)

    windows = skimage.util.view_as_windows(band_data, (5,32,128), step=128).squeeze()
    # (5, 32, 60, 128)
    windows = rearrange(windows, 'a b c d -> b c a d')
    
    if feature == 'psd':
        features = scipy.signal.periodogram(windows)[1]
        features = np.mean(features, axis=-1)
    elif feature == 'de':
        features = np.apply_along_axis(calculate_de, -1, windows)

    
    features = rearrange(features, 'a b c -> (a b) c')
    features = torch.FloatTensor(features)

    return features

In [4]:
def plot_video(signal_data):
    electrodes = Electrodes(expand_3d=False)
    fig, axs = plt.subplots(32, sharex=True, figsize=(20,50))
    fig.tight_layout()
    video = signal_data[0]
    for i in range(32):
        c = [float(i)/float(32), 0.0, float(32-i)/float(32)] #R,G,B
        axs[i].set_title(f'{electrodes.channel_names[i]}', loc='left', fontsize=20)
        axs[i].plot(video[i],color=c)
        axs[i].spines['top'].set_visible(False)
        axs[i].spines['right'].set_visible(False)
        axs[i].spines['left'].set_visible(False)
        axs[i].spines['bottom'].set_visible(False)
    plt.savefig('eeg.png')
    

In [5]:
def remove_baseline_mean(signal_data):
    # Take first three senconds of data
    signal_baseline = np.array(signal_data[:,:,:128*3]).reshape(40,32,128,-1)
    # Mean of three senconds of baseline will be deducted from all windows
    signal_noise = np.mean(signal_baseline,axis=-1)
    # Expand mask
    signal_noise = repeat(signal_noise,'a b c -> a b (d c)',d=60)
    return signal_data[:,:,128*3:] - signal_noise

In [6]:
def process_video_wavelet(video, feature='energy', time_domain=False):
    band_widths = [32,16,8,4]
    features = []
    for i in range(5):
        if i == 0:
            # Highest frequencies (64-128Hz) are not used
            cA, cD = pywt.dwt(video.numpy(), 'db4')
        else:
            cA, cD = pywt.dwt(cA, 'db4')
            cA_windows = skimage.util.view_as_windows(cA, (32,band_widths[i-1]*2), step=band_widths[i-1]).squeeze()
            cA_windows = np.transpose(cA_windows[:59,:,:],(1,0,2))
            if feature == 'energy':
                cA_windows = np.square(cA_windows)
                cA_windows = np.sum(cA_windows, axis=-1)
                features.append(cA_windows)
                
    if time_domain:
        features = np.transpose(features,(2,1,0))
    features = rearrange(features, 'a b c -> (a b) c')
    features = torch.FloatTensor(features)
    return features

In [7]:
# Constants used to define data paths
ROOT_DIR = './'
RAW_DIR = 'data/matlabPREPROCESSED'
PROCESSED_DIR = 'data/graphProcessedData'

dataset = DEAPDatasetEEGFeatures(root= ROOT_DIR, raw_dir= RAW_DIR, processed_dir= PROCESSED_DIR, feature='wav')
# dataset = dataset.shuffle()
dataset[0]

Using global connections


Data(edge_attr=[776], edge_index=[2, 776], x=[128, 59], y=[1, 4])

In [8]:
# 880 used for training, 220 validation and 180 testing
test_participant = 0
# 
# splt_idx = 35

# 85% used for train/val
train_dataset = dataset[0:test_participant*40] + dataset[test_participant*40+40:]
test_dataset = dataset[test_participant*40:test_participant*40+40]

len(train_dataset),len(test_dataset)

(1240, 40)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print(device)

cuda


In [75]:
from torch_geometric.nn import GCN2Conv, GCNConv
class Model(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels=512, cnn_hidden_dim = 256):
        super(Model, self).__init__()
        
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        
        self.gconv1 = GCN2Conv(in_channels,1)
        self.gconv2 = GCNConv(in_channels,hidden_channels)
        
#         self.gconv3 = GCNConv(in_channels,hidden_channels)
        
        # self.rnn = torch.nn.GRU(hidden_channels, rnn_hidden_dim, 2,dropout=0.2, batch_first=True)
        self.cnn1 = torch.nn.Conv1d(4*hidden_channels, hidden_channels, kernel_size=1, stride=1)
        self.cnn2 = torch.nn.Conv1d(hidden_channels, cnn_hidden_dim, kernel_size=1, stride=1)
        
        self.lin1 = torch.nn.Linear(32*cnn_hidden_dim, 1)
#         self.lin2 = torch.nn.Linear(cnn_hidden_dim, 1)

        
    def forward(self, batch):
        bs = len(torch.unique(batch.batch))
        x_, edge_index, edge_attr = batch.x, batch.edge_index, batch.edge_attr
#         print(x.shape)
        x = self.gconv1(x_,x_, edge_index, edge_attr)
#         x = x.relu()
        x = self.gconv2(x, edge_index, edge_attr)
        x = x.tanh()
        
#         x = self.gconv3(x, edge_index, edge_attr )
#         print('-0----------------------------------------------------')
#         print(x)
#         x = x.relu()
#         print(x)
#         x = F.dropout(x, p=0.5, training=self.training)
        
        # x = rearrange(x, '(bs a b) c -> (bs b) a c', bs=bs, b=32, c=self.hidden_channels)
        # o, (h_n,c_n) = self.rnn(x)
        x = rearrange(x, '(bs g e) f -> bs (g f) e', bs=bs, e=32)
        x = self.cnn1(x)
        x = x.relu()
        x = self.cnn2(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = rearrange(x, 'bs a b -> bs (a b)', bs=bs)
        
        
        x = self.lin1(x)
#         x = x.relu()
#         x = self.lin2(x)
        x = x.view(-1)
        return x.sigmoid()

In [76]:
# %%timeit

model = Model(train_dataset[0].x.shape[1]).to(device)  
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f'Model parameter count: {pytorch_total_params}')

# model = model.to(device)
# optimizer = torch.optim.Adadelta(model.parameters(), lr=.1, rho=0.9, eps=1e-06, weight_decay=1e-5)
# optimizer = torch.optim.SGD(model.parameters(),lr=1e-4, weight_decay=1e-4)
# optimizer = torch.optim.Adam(model.parameters(),lr=1e-4, weight_decay=1e-2)
# optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-5)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=5e-4)
optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-4, lr_decay=0, weight_decay=5e-2)

# Instantiate optimizer
# scheduler = StepLR(optimizer, step_size=20, gamma=0.7)

# criterion = nn.MSELoss()
criterion = nn.BCELoss()



def train(loader, target = 0):
    model.train()
    losses = []
    right = 0
    tot = 0
    for batch in loader:
        optimizer.zero_grad()
        batch = batch.to(device)
        y = (batch.y[:,target] > 5).float()
#         y = batch.y[:,target]
        out = model(batch)
        loss = criterion(out,y)
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        right += torch.eq(out > .5, y > .5).sum().item()
        tot += y.shape[0]
    return np.array(losses).mean(), right/tot

def test(loader,verbose=False, target = 0):
    model.eval()
    losses = []
    right = 0
    tot = 0
    for batch in loader:
        batch = batch.to(device)
        y = (batch.y[:,target] > 5).float()
#         y = batch.y[:,target]
        out = model(batch)
        if verbose:
            print(out,y)
        loss = criterion(out,y)
        losses.append(loss.item())
        right += torch.eq(out > .5, y > .5).sum().item()
        tot += y.shape[0]
    return np.array(losses).mean(), right/tot

best_val_loss = np.inf
esp = 0
MAX_ESP = 50

BS = 8

target = 0 # Valence-Arousal-Dominance-Liking

splt_idx = 1160
train_data, val_data = torch.utils.data.random_split(train_dataset, [splt_idx, len(train_dataset)-splt_idx])

train_loader = DataLoader(train_data, batch_size=BS, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BS)
for epoch in range(1, 10000):    

    # Training and validation
    train_loss, train_acc = train(train_loader, target = target)
    val_loss, val_acc = test(val_loader , target = target)
    print(f'Epoch {epoch};t loss: {train_loss:.5f} ;t acc: {train_acc:.2f} ;v loss: {val_loss:.5f} ;v acc: {val_acc:.2f}')

    # Early stopping and checkpoint
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        esp = 0
        torch.save(model.state_dict(),'./best_params') 
    else:
        esp += 1
        if esp >= MAX_ESP:
            break
            
    if epoch % 20 == 0:
        test_loader = DataLoader(test_dataset, batch_size=1)
        loss, acc = test(test_loader, True)
        print(f'Test loss: {loss} ; Test acc: {acc}')


print('Finished training')

Model parameter count: 1222810
Epoch 1;t loss: 0.68884 ;t acc: 0.53 ;v loss: 0.67046 ;v acc: 0.68
Epoch 2;t loss: 0.67960 ;t acc: 0.57 ;v loss: 0.66599 ;v acc: 0.66
Epoch 3;t loss: 0.67678 ;t acc: 0.58 ;v loss: 0.66223 ;v acc: 0.66
Epoch 4;t loss: 0.67245 ;t acc: 0.60 ;v loss: 0.65808 ;v acc: 0.66
Epoch 5;t loss: 0.66831 ;t acc: 0.58 ;v loss: 0.66561 ;v acc: 0.66
Epoch 6;t loss: 0.66621 ;t acc: 0.61 ;v loss: 0.65902 ;v acc: 0.66
Epoch 7;t loss: 0.66478 ;t acc: 0.61 ;v loss: 0.66442 ;v acc: 0.65
Epoch 8;t loss: 0.66428 ;t acc: 0.61 ;v loss: 0.66695 ;v acc: 0.62
Epoch 9;t loss: 0.66279 ;t acc: 0.62 ;v loss: 0.66779 ;v acc: 0.61
Epoch 10;t loss: 0.65935 ;t acc: 0.64 ;v loss: 0.66226 ;v acc: 0.61
Epoch 11;t loss: 0.65740 ;t acc: 0.63 ;v loss: 0.66133 ;v acc: 0.61
Epoch 12;t loss: 0.65719 ;t acc: 0.64 ;v loss: 0.65996 ;v acc: 0.62
Epoch 13;t loss: 0.65629 ;t acc: 0.64 ;v loss: 0.65723 ;v acc: 0.64
Epoch 14;t loss: 0.65227 ;t acc: 0.64 ;v loss: 0.66089 ;v acc: 0.62
Epoch 15;t loss: 0.65336 ;

Epoch 41;t loss: 0.62409 ;t acc: 0.67 ;v loss: 0.66485 ;v acc: 0.61
Epoch 42;t loss: 0.62405 ;t acc: 0.68 ;v loss: 0.66263 ;v acc: 0.60
Epoch 43;t loss: 0.62269 ;t acc: 0.68 ;v loss: 0.66418 ;v acc: 0.61
Epoch 44;t loss: 0.62189 ;t acc: 0.68 ;v loss: 0.66286 ;v acc: 0.60
Epoch 45;t loss: 0.61995 ;t acc: 0.69 ;v loss: 0.66317 ;v acc: 0.60
Epoch 46;t loss: 0.61871 ;t acc: 0.68 ;v loss: 0.66474 ;v acc: 0.60
Epoch 47;t loss: 0.61916 ;t acc: 0.69 ;v loss: 0.67263 ;v acc: 0.57
Epoch 48;t loss: 0.61747 ;t acc: 0.68 ;v loss: 0.66411 ;v acc: 0.59
Epoch 49;t loss: 0.61789 ;t acc: 0.69 ;v loss: 0.66972 ;v acc: 0.59
Epoch 50;t loss: 0.61551 ;t acc: 0.68 ;v loss: 0.65979 ;v acc: 0.57
Epoch 51;t loss: 0.61524 ;t acc: 0.69 ;v loss: 0.66168 ;v acc: 0.59
Epoch 52;t loss: 0.61432 ;t acc: 0.70 ;v loss: 0.66072 ;v acc: 0.57
Epoch 53;t loss: 0.61466 ;t acc: 0.68 ;v loss: 0.66226 ;v acc: 0.60
Epoch 54;t loss: 0.61239 ;t acc: 0.70 ;v loss: 0.67003 ;v acc: 0.57
Epoch 55;t loss: 0.61383 ;t acc: 0.69 ;v loss: 0

Epoch 81;t loss: 0.59060 ;t acc: 0.72 ;v loss: 0.66282 ;v acc: 0.57
Epoch 82;t loss: 0.58834 ;t acc: 0.72 ;v loss: 0.66213 ;v acc: 0.57
Epoch 83;t loss: 0.58779 ;t acc: 0.73 ;v loss: 0.66826 ;v acc: 0.56
Finished training


In [77]:
model.load_state_dict(torch.load('./best_params'))
test_loader = DataLoader(test_dataset, batch_size=1)
loss, acc = test(train_loader, False,target=target)
print(f'Train loss: {loss} ; Train acc: {acc}')
loss, acc = test(val_loader, False,target=target)

print(f'Val loss: {loss} ; Val acc: {acc}')
loss, acc = test(test_loader, True,target=target)
print(f'Test loss: {loss} ; Test acc: {acc}')

# TODO: scheduler(?) Loss/acc records

Train loss: 0.628433863253429 ; Train acc: 0.6732758620689655
Val loss: 0.6556640923023224 ; Val acc: 0.6
tensor([0.6001], device='cuda:0', grad_fn=<SigmoidBackward>) tensor([1.], device='cuda:0')
tensor([0.6186], device='cuda:0', grad_fn=<SigmoidBackward>) tensor([1.], device='cuda:0')
tensor([0.5911], device='cuda:0', grad_fn=<SigmoidBackward>) tensor([1.], device='cuda:0')
tensor([0.6067], device='cuda:0', grad_fn=<SigmoidBackward>) tensor([0.], device='cuda:0')
tensor([0.4521], device='cuda:0', grad_fn=<SigmoidBackward>) tensor([1.], device='cuda:0')
tensor([0.4755], device='cuda:0', grad_fn=<SigmoidBackward>) tensor([1.], device='cuda:0')
tensor([0.5078], device='cuda:0', grad_fn=<SigmoidBackward>) tensor([1.], device='cuda:0')
tensor([0.5390], device='cuda:0', grad_fn=<SigmoidBackward>) tensor([1.], device='cuda:0')
tensor([0.6292], device='cuda:0', grad_fn=<SigmoidBackward>) tensor([0.], device='cuda:0')
tensor([0.4726], device='cuda:0', grad_fn=<SigmoidBackward>) tensor([0.], d